In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/human-activity-recognition-with-smartphones/train.csv
/kaggle/input/human-activity-recognition-with-smartphones/test.csv


In [2]:
df=pd.read_csv('/kaggle/input/human-activity-recognition-with-smartphones/train.csv')
dftest=pd.read_csv('/kaggle/input/human-activity-recognition-with-smartphones/test.csv')


In [3]:
df.columns=df.columns.str.strip()
dftest.columns=dftest.columns.str.strip()

In [4]:
# let us map the Activity to numerical
assign={
    'WALKING':0,
    'STANDING':1,
    'LAYING':2,
    'WALKING_UPSTAIRS':3,
    'SITTING':4,
    'WALKING_DOWNSTAIRS':5
}
df['Activity_num']=df['Activity'].map(assign)
dftest['Activity_num']=dftest['Activity'].map(assign)

In [5]:
df.dropna(inplace=True)
dftest.dropna(inplace=True)

In [6]:
df.drop(columns=['Activity'],inplace=True,axis=1)
dftest.drop(columns=['Activity'],inplace=True,axis=1)

In [7]:
import pandas as pd
import numpy as np

# Assume df is already defined and remains consistent across executions

# Compute the correlation matrix
correlation_matrix = df.corr()

# Get the correlations of 'Activity_num' with all other columns
activity_corr = correlation_matrix['Activity_num']

# Drop the self-correlation
activity_corr = activity_corr.drop(labels=['Activity_num'])

# Sort the correlations in descending order
sorted_activity_corr = activity_corr.sort_values(ascending=False)

# Round the correlations to two decimal places
rounded_corr = sorted_activity_corr.round(2)

# Initialize an empty dictionary to store unique correlations
unique_corr = {}
unique_values = set()

# Iterate over the rounded correlations to keep only unique ones
for col, corr_value in rounded_corr.items():
    rounded_value = round(corr_value, 2)
    if rounded_value not in unique_values:
        unique_corr[col] = rounded_value
        unique_values.add(rounded_value)

# Convert the dictionary back to a Series for better display
unique_corr_series = pd.Series(unique_corr)
count = unique_corr_series.count()

print(count)


59


In [8]:
unique_columns = list(unique_corr.keys())
X_train=df[unique_columns]

In [9]:
X_test=dftest[unique_columns]

In [10]:
y_train=df['Activity_num']
y_test=dftest['Activity_num']

In [11]:
# prompt: want to import Logisitc regression,decision tree classifier,svc,random forest classifier,adaboost,gradient boost,xgboost classifier , accuracy score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [12]:
# let us apply XGBClassifier
xgbc=XGBClassifier()
xgbc.fit(X_train,y_train)
y_pred7=xgbc.predict(X_test)
accuracy_score(y_pred7,y_test)

0.9311163895486936

In [13]:
# got maximum accuracy in xbbclassifer let us now hyper tune the parameter and again see the accuracy
from sklearn.model_selection import  GridSearchCV

# **Using RandomizedSearchCV for hypertuning the parameters**

In [14]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import RandomizedSearchCV
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score

# # Example parameter grid with reduced size
# param_grid = {
#     'n_estimators': [100, 300, 500, 1000],
#     'learning_rate': [0.01, 0.1, 0.3],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0],
#     'gamma': [0, 0.1, 0.3]
# }

# xgbc = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# random_search = RandomizedSearchCV(estimator=xgbc, param_distributions=param_grid, n_iter=50, cv=3, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42)

# # Assuming X_train and y_train are your training data
# random_search.fit(X_train, y_train)

# # Predict on the test set
# y_pred = random_search.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.4f}")

# # Print the best parameters
# best_params = random_search.best_params_
# print("Best parameters found: ", best_params)


In [15]:
xgbc_beta = XGBClassifier(subsample= 0.8, n_estimators= 300, max_depth=5, learning_rate= 0.1, gamma= 0, colsample_bytree= 1.0,use_label_encoder=False, eval_metric='mlogloss')
xgbc_beta.fit(X_train, y_train)
y_predb = xgbc_beta.predict(X_test)
accuracy = accuracy_score(y_test, y_predb)
accuracy

0.9362063115032236

#  Getting almost 94% accuracy

In [16]:
# let us create a pickel file and save it
import pickle
with open('xgbc_model_beta.pkl', 'wb') as file:
    pickle.dump(xgbc_beta, file)